In [86]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoost,CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
import sklearn.metrics  
import math
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jasondata/train.json
/kaggle/input/cmrvdbtae/sample_submission.csv
/kaggle/input/cmrvdbtae/train.csv
/kaggle/input/cmrvdbtae/test.csv


# Read_Data

In [87]:
train=pd.read_csv('/kaggle/input/cmrvdbtae/train.csv')
test=pd.read_csv('/kaggle/input/cmrvdbtae/test.csv')
Id=test.iloc[:,1]

# Preprocessing

In [88]:
# label_encoding
sequence_encmap = {'A': 0, 'G' : 1, 'C' : 3, 'U' : 2}
structure_encmap = {'.' : 0, '(' : 1, ')' : 1}
looptype_encmap = {'S':6, 'E':2, 'H':0, 'I':5, 'X':4, 'M':3, 'B':1}

enc_targets = ['sequence', 'a1_sequence', 'a2_sequence', 'a3_sequence',
               'b1_sequence', 'b2_sequence', 'b3_sequence','b4_sequence',
               'a4_sequence','b5_sequence','a5_sequence','structure','a1_structure',
               'b1_structure','b2_structure','a2_structure','b3_structure','a3_structure',
               'b4_structure','a4_structure','a5_structure','b5_structure','predicted_loop_type',
               'b1_predicted_loop_type','a1_predicted_loop_type','b2_predicted_loop_type','a2_predicted_loop_type',
               'b3_predicted_loop_type','a3_predicted_loop_type','b4_predicted_loop_type','a4_predicted_loop_type',
               'b5_predicted_loop_type','a5_predicted_loop_type','predicted_loop_type'
               ]
enc_maps = [sequence_encmap, sequence_encmap, sequence_encmap, sequence_encmap,
            sequence_encmap, sequence_encmap, sequence_encmap,
            sequence_encmap, sequence_encmap, sequence_encmap,sequence_encmap,
            structure_encmap, structure_encmap, structure_encmap, structure_encmap,
            structure_encmap, structure_encmap, structure_encmap,structure_encmap,structure_encmap,structure_encmap,structure_encmap,
            looptype_encmap, looptype_encmap, looptype_encmap,
            looptype_encmap, looptype_encmap, looptype_encmap,
            looptype_encmap,looptype_encmap,looptype_encmap,looptype_encmap,looptype_encmap]

for t, m in zip(enc_targets, enc_maps):
    train[t] = train[t].apply(lambda x: m[x] if x in m else -1)
    test[t] = test[t].apply(lambda x: m[x] if x in m else -1)

In [89]:
# p=train["id_seqpos"].copy()

In [90]:
y = train[['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C']]
X = train.drop(y, axis =1)
X=X.drop(['id','id_seqpos'],axis=1)
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [91]:
def rmse(y_actual, y_pred):
    mse = sklearn.metrics.mean_squared_error(y_actual, y_pred)  
    rmse = math.sqrt(mse)  
    return rmse

def mcrmse(y_actual, y_pred):
    score=0
    num_scored=3
    for i in range(num_scored):
        score += rmse(y_actual.iloc[:, i], y_pred[:,  i]) / num_scored
    return score

In [76]:
def pos(id_seq):
    z=id_seq[-2:]
    if z.isdigit():
        return z
    else:
        return z[-1]
X["position"]=0
p=pd.DataFrame(p)
X["position"]=p["id_seqpos"].apply(lambda x:pos(x))
X["position"]=X["position"].astype("int")


In [93]:
X

,sequence,structure,predicted_loop_type,reactivity_error,deg_error_Mg_pH10,deg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_50C,deg_error_50C,...,b4_structure,a4_structure,b4_predicted_loop_type,a4_predicted_loop_type,b5_sequence,a5_sequence,b5_structure,a5_structure,b5_predicted_loop_type,a5_predicted_loop_type
0,0,0,2,0.1195,0.0790,0.3680,0.0851,0.0792,0.5685,0.1051,...,1,1,0,0,2,0,0,1,3,0
1,1,0,3,0.1033,0.0799,0.0878,0.0878,0.0683,0.0666,0.0864,...,1,0,0,2,2,2,0,0,6,2
2,1,1,0,0.1375,0.0744,0.1835,0.1969,0.1163,0.0000,0.1088,...,1,1,0,0,0,1,1,1,0,0
3,1,1,0,0.0471,0.1075,0.0657,0.0615,0.0506,0.0749,0.0913,...,0,1,1,0,0,3,0,1,1,0
4,3,1,0,0.1352,0.2168,0.3375,0.2623,0.1569,0.6247,0.3370,...,1,1,0,0,1,1,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130555,3,1,0,0.0393,0.0728,0.1479,0.0493,0.0481,0.4711,0.0883,...,1,1,0,0,0,0,0,0,2,5
130556,0,0,4,0.0351,0.0583,0.0580,0.0660,0.0419,0.2559,0.1039,...,0,0,2,4,0,3,0,0,2,4
130557,0,0,2,0.0601,0.0515,0.2838,0.0793,0.0528,0.3089,0.0861,...,0,0,2,3,3,3,0,1,2,0
130558,1,0,3,0.1063,0.0738,0.1885,0.1276,0.0773,0.1001,0.1234,...,1,0,0,3,1,1,1,1,0,0


In [94]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 79)

# Modeling

In [78]:
%time
xgb = MultiOutputRegressor(XGBRegressor(learning_rate=0.2,n_estimators=1500,max_depth=4,min_child_weight = 1,gamma=0,nthread =4,scale_pos_weight =1,seed =35))
xgb.fit(X_train, y_train)
yhat = xgb.predict(X_val)
c=mcrmse(y_val, yhat)
print(c)


CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.58 µs
0.3186949081061241


In [79]:
%time
gd = MultiOutputRegressor(GradientBoostingRegressor())
gd.fit(X_train, y_train)
yhat = gd.predict(X_val)
c=mcrmse(y_val, yhat)
print(c)


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs
0.44690694967424494


In [80]:
%time
cb = MultiOutputRegressor(CatBoostRegressor())
cb.fit(X_train, y_train)
yhat = cb.predict(X_val)
c=mcrmse(y_val, yhat)
print(c)


CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 10.3 µs
Learning rate set to 0.085343
0:	learn: 0.7333953	total: 28.4ms	remaining: 28.4s
1:	learn: 0.7173979	total: 47ms	remaining: 23.5s
2:	learn: 0.7025208	total: 69.5ms	remaining: 23.1s
3:	learn: 0.6900567	total: 86.1ms	remaining: 21.4s
4:	learn: 0.6779541	total: 110ms	remaining: 21.8s
5:	learn: 0.6676663	total: 126ms	remaining: 20.9s
6:	learn: 0.6577881	total: 145ms	remaining: 20.6s
7:	learn: 0.6491706	total: 162ms	remaining: 20.1s
8:	learn: 0.6414805	total: 178ms	remaining: 19.6s
9:	learn: 0.6342745	total: 196ms	remaining: 19.4s
10:	learn: 0.6279543	total: 212ms	remaining: 19s
11:	learn: 0.6227319	total: 229ms	remaining: 18.8s
12:	learn: 0.6175911	total: 244ms	remaining: 18.5s
13:	learn: 0.6128536	total: 256ms	remaining: 18.1s
14:	learn: 0.6082060	total: 271ms	remaining: 17.8s
15:	learn: 0.6041540	total: 291ms	remaining: 17.9s
16:	learn: 0.6005799	total: 311ms	remaining: 18s
17:	learn: 0.5972375	total: 326ms	remaining: 17.8s


In [81]:
%time
rf = MultiOutputRegressor(RandomForestRegressor())
rf.fit(X_train, y_train)
yhat = rf.predict(X_val)
c=mcrmse(y_val, yhat)
print(c)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs
0.43214791503636296


In [82]:
%time
rf = MultiOutputRegressor(AdaBoostRegressor())
rf.fit(X_train, y_train)
yhat = rf.predict(X_val)
c=mcrmse(y_val, yhat)
print(c)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs
1.1329980462440639


In [83]:
%time
rf = MultiOutputRegressor(KNeighborsRegressor())
rf.fit(X_train, y_train)
yhat = rf.predict(X_val)
c=mcrmse(y_val, yhat)
print(c)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs
0.5705327361010517


In [84]:
%time
rf = MultiOutputRegressor(LinearRegression())
rf.fit(X_train, y_train)
yhat = rf.predict(X_val)
c=mcrmse(y_val, yhat)
print(c)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs
0.5964142257355979


In [85]:
%time
rf = MultiOutputRegressor(LGBMRegressor())
rf.fit(X_train, y_train)
yhat = rf.predict(X_val)
c=mcrmse(y_val, yhat)
print(c)

CPU times: user 13 µs, sys: 0 ns, total: 13 µs
Wall time: 7.15 µs
0.4397519020339189


In [95]:
"""from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

r=GridSearchCV(cv=5,estimator=MultiOutputRegressor(estimator=XGBRegressor( base_score=None,
                                                                        booster=None,
                                                                        callbacks=None,
                                                                        colsample_bylevel=None,
                                                                        colsample_bynode=None,
                                                                        colsample_bytree=0.75,
                                                                        early_stopping_rounds=None,
                                                                        eval_metric=None,
                                                                        gamma=None,
                                                                        gpu_id=None,
                                                                        frow_policy=None,
                                                                        importance_type=None,
                                                                        interaction_constraints=None,
                                                                        max_leaves=None,
                                                                        min_child_wight=None,
                                                                        monotone_constraints=None,
                                                                        n_estimators=100,
                                                                        n_jobs=None,
                                                                        num_parallel_tree=None,
                                                                        predictor=None,
                                                                        random_state=79,
                                                                        reg_alpha=1,
                                                                        reg_lambda=2)),
            n_jobs=-1,
            param_grid={'estimator__learning_rate':[0.1,0.2,0.3],
                      'estimator__max_depth':[3,4,5],
                      'estimator__n_estimators':[800,900,100]},
return_train_score=True)
r.fit(X_train, y_train)
"""

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
params = {
        'estimator__sampling_method':['uniform','subsample','gradient_based'],
        'estimator__min_child_weight': [1, 5, 10],
        'estimator__gamma': [0.5, 1, 1.5,2.5,4,5],
        'estimator__subsample': [0.5,0.6, 0.8, 1.0],
        'estimator__colsample_bytree': [0.6, 0.8, 1.0],
        'estimator__max_depth': [3, 4, 5,6,7],
        'estimator__learning_rate':[0.1,0.3,0.5,0.8]
        }


regr=GridSearchCV(MultiOutputRegressor(xgb), param_grid=params )
regr.fit(X_train, y_train)
yhat = regr.predict(X_val)
c=mcrmse(y_val, yhat)
print(c)


# Test

In [33]:
p2=test["id_seqpos"].copy()
p2=pd.DataFrame(p2)
test["position"]=0
test["position"]=p2["id_seqpos"].apply(lambda x:pos(x))
test["position"]=test["position"].astype("int")


'p2=test["id_seqpos"].copy()\np2=pd.DataFrame(p2)\ntest["position"]=0\ntest["position"]=p2["id_seqpos"].apply(lambda x:pos(x))\ntest["position"]=test["position"].astype("int")\n'

In [55]:
test=test.drop(['id','id_seqpos'],axis=1)


In [61]:
y_final = regr.predict(test)

In [62]:
sample_submission=pd.DataFrame()
sample_submission['id_seqpos']=Id
sample_submission['reactivity'] = y_final[:,0]
sample_submission['deg_Mg_pH10'] = y_final[:,1]
sample_submission['deg_Mg_50C'] = y_final[:,2]
sample_submission.to_csv("/kaggle/working/sub42.csv")
